In [1]:
from __future__ import annotations
from utils_to_build_QAOAAnsatz import *

from random_instances import info_dim6 as info
# from Wang_instances import info_dim6 as info

In [2]:
n = 6
instances = [4]

for instance in instances:
    print(f"\n------------------")
    print(f"--- Instance {instance} ---")
    print(f"------------------")
    
    # Get and sort exact covers by increasing number of '1's
    ec = info[instance]["exact_covers"]
    ec = sorted(ec, key=lambda x: x.count('1'))
    L_list = [e.count("1") - 1 for e in ec]

    # Define the problem instance
    U, subsets_dict = define_instance(n, instance, verbose=False)
    gap_list = []

    for L in L_list:
        print(f"\n--- L = {L} ---")
        k = k_from_L(n, instance, L)

        # Compute the feasible spectrum
        _, _, states_feas, energies_feas, _ = find_spectrum(U, subsets_dict, n, k)

        # Print energies of MEC and exact covers
        print("  Energies of MEC and exact covers:")
        for e in ec:
            energy = compute_energy_Wang(e, U, subsets_dict, k)
            print(f"  Energy({e}) = {energy}")

        # Sort states and energies
        sorted_pairs = sorted(zip(energies_feas, states_feas))
        energies_feas_sorted = [energy for energy, _ in sorted_pairs]
        states_feas_sorted = [state for _, state in sorted_pairs]

        # Check if ground state (GS) is the MEC
        if states_feas_sorted[0] == ec[0]:
            print("\n  ** THE GS IS THE MEC **")
        else:
            print("\n  ### THE GS IS NOT THE MEC ###")
        print(f"  GS: Energy({states_feas_sorted[0]}) = {energies_feas_sorted[0]}\n")

        # Check if the first excited state is an exact cover (but not the MEC)
        if states_feas_sorted[1] != ec[0] and states_feas_sorted[1] in ec:
            print("  ** THE 1st excited IS AN EXACT COVER **")
        else:
            print("  ### THE 1st excited IS NOT AN EXACT COVER ###")
        print(f"  1st excited: Energy({states_feas_sorted[1]}) = {energies_feas_sorted[1]}\n")

        # Compute energy gap
        gap_gs_excited = abs(energies_feas_sorted[0]) - abs(energies_feas_sorted[1])
        print(f"\n  The gap GS-1st_excited is {gap_gs_excited}.")

        total_energy = abs(min(energies_feas) - max(energies_feas))
        print("  gap GS-1st_excited / total energy [%]:", gap_gs_excited / total_energy * 100)

        gap_list.append(gap_gs_excited)



------------------
--- Instance 4 ---
------------------

--- L = 2 ---
  Energies of MEC and exact covers:
  Energy(100011) = -0.1
  Energy(101011) = -0.11428571428571428

  ### THE GS IS NOT THE MEC ###
  GS: Energy(101011) = -0.11428571428571428

  ### THE 1st excited IS NOT AN EXACT COVER ###
  1st excited: Energy(001011) = -0.1


  The gap GS-1st_excited is 0.014285714285714277.
  gap GS-1st_excited / total energy [%]: 12.499999999999993

--- L = 3 ---
  Energies of MEC and exact covers:
  Energy(100011) = -0.17142857142857143
  Energy(101011) = -0.19999999999999998

  ### THE GS IS NOT THE MEC ###
  GS: Energy(101011) = -0.19999999999999998

  ### THE 1st excited IS NOT AN EXACT COVER ###
  1st excited: Energy(001011) = -0.17142857142857143


  The gap GS-1st_excited is 0.028571428571428553.
  gap GS-1st_excited / total energy [%]: 14.285714285714276
